# Este notebook muestra como hacer una consulta de datos a Properati Data en Google Big Query

Para ello tienen que tener previamente una credencial de google big query, en la forma de un archivo json, en el directorio del projecto.
Para más información sobre la fuente de datos ver el sitio de [Properati Data](https://www.properati.com.ar/data/)


In [100]:
#Autor Ricardo Pasquini
import os, inspect
from google.cloud import bigquery
from google.oauth2 import service_account

In [116]:
import pandas as pd
from pandas.io.json import json_normalize

In [3]:
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))

In [4]:
currentdir

'C:\\Users\\Richard\\Drive\\projects\\geotests'

## Generar una clave
Para generar su propia clave y descargarla en formato json sigan las siguientes instrucciones: 
https://cloud.google.com/docs/authentication/getting-started

Cargo mi credencial

In [5]:
credentials = service_account.Credentials.from_service_account_file(
    currentdir+'\credentialsbigquery\NOMBREDELALLAVE.json')

In [61]:
project_id = 'dataproperati2019'
client = bigquery.Client(credentials= credentials,project=project_id)


# Aqui se define el pedido en formato SQL.
QUERY = (
'SELECT * FROM `properati-dw.public.ads` WHERE start_date >= "2019-05-01" AND start_date <= "2020-05-15" AND type = "Propiedad" AND country = "Argentina" '
'AND place.l1 = "Argentina" '
#'AND property.type = "Departamento" '
'AND property.operation = "Alquiler" '
'AND property.surface_total > 0 '
'AND property.surface_covered > 0 '
'AND property.price > 0 '
'AND (place.l2="Capital Federal" or place.l2="Buenos Aires Interior" or place.l2="Bs.As. G.B.A. Zona Sur" or place.l2="Bs.As. G.B.A. Zona Norte" or place.l2="Bs.As. G.B.A. Zona Oeste") '
)

query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

#for row in rows:
#    print(row.start_date, row.country)
    

Realizando el query y guardando en un dataframe

In [62]:
df=client.query(QUERY).to_dataframe()

Esta celda lleva el campo property a columnas y las combina con el resto columnas 

In [112]:
df2=pd.concat([df[['type', 'type_i18n', 'country', 'id', 'start_date', 'end_date',
       'created_on', 'place', 'development']], json_normalize(df.property), json_normalize(df.place)], axis=1)

In [97]:
df2.description[0]

'¡ MONOAMBIENTE DIVISIBLE ! ¡ SIN EXPENSAS ! <br>#Alquiler | #Departamento | #Boedo | #1AMB | #PH | #Balcón<br>Avenida San Juan y Avenida Boedo - Ciudad Autónoma de Bs As <br><br>#Características: <br>~ PH Sin Expensas<br>~ Acceso Primer Piso por Escalera.<br>~ Ambiente Dividido Bien Distribuido.<br>~ Cocina Integrada con Mobiliario a Estrenar.<br>~ Espacioso Comedor..<br>~ Dormitorio con Balcón al Frente. <br>~ Baño Completo con Bañera..<br>~ Reciclado a Nuevo.<br>~ Aire Acondicionado Split marca LG a Estrenar.<br>~ Calefón a Gas a Estrenar .<br>~ Cocina a Gas a Estrenar.<br>~ Lavarropas  a Estrenar.<br>~ Pisos de: Cerámico. <br>~Totalmente Pintada. <br>~ Excelente Ubicación sobre Avenida San Juan en Pleno Centro del Barrio de Boedo. <br>~ No Apto Mascotas. <br><br>#MásInfo sobre este aviso en el siguiente link: <br>http:// El edificio no cuenta con accesibilidad para personas con movilidad reducida  <br>#Consultas a <br>#Oficina en Avenida San Juan 3511 - CABA <br><br>Desde 1921 - #O

In [114]:
df2.columns

Index(['type', 'type_i18n', 'country', 'id', 'start_date', 'end_date',
       'created_on', 'place', 'development', 'operation', 'operation_i18n',
       'type', 'type_i18n', 'rooms', 'bedrooms', 'bathrooms', 'surface_total',
       'surface_covered', 'price', 'currency', 'price_period', 'title',
       'description', 'lat', 'lon', 'l1', 'l2', 'l3', 'l4', 'l5', 'l6'],
      dtype='object')

# Descargar a un CSV

In [115]:
df2.to_csv('data\\descargatest2020\\test1.csv')

# Preparar geodataframe

In [ ]:
!pip install geopandas
import geopandas as gpd
crs = {'init' :'epsg:4326'}
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(df.lon, df.lat)]
gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)

# Exportar a un shapefile

In [ ]:
gdf.to_file("data\\descargatest2020\\test1.shp")

# Ejemplo insertar a un MongoDB  local

In [ ]:
import pymongo

client = pymongo.MongoClient('localhost', 27017)
db = client["properati"]  #vivienda is the name of the db

In [9]:
df2=df.copy()
df2.start_date=pd.to_datetime(df.start_date)
df2.end_date=pd.to_datetime(df.end_date, errors = 'coerce') # convierto a nulos los que no se corresponden con fecha verdadera
df2.created_on=pd.to_datetime(df.created_on)
df2=df2[~pd.isnull(df2.end_date)]

In [10]:
dfdictionary=df2.to_dict('records')

In [11]:
db.raw.insert_many(dfdictionary)